# Split dataset and upload to huggingface
* This notebook will spilt the dataset into: training, validate and test set.
* Then upload the dataset to huggingface 

In [2]:
import pandas as pd
import numpy as np
from datasets import Dataset, DatasetDict
from huggingface_hub import notebook_login

In [3]:
name = "highest_and_high_vs_rest"
file_name = f"clean_{name}.csv"
df = pd.read_csv(file_name)
df

,text_clean,label
0,i have the same problem as here but i update f...,1
1,the time estimate and time remaining values on...,0
2,note this bug report is for confluence server ...,1
3,steps to reproduce get a rapid view which is r...,0
4,i used the feed builder to create an rss feed ...,0
...,...,...
28851,issue summary connect app ui elements with the...,1
28852,issue summary group picker with a restricted c...,0
28853,navigation navigate to file status working cop...,1
28854,note this bug report is for confluence cloud u...,0


In [4]:
null_rows = df[df['text_clean'].isnull()]
null_rows

,text_clean,label


In [5]:
df = df.dropna(subset=['text_clean'])
# reset index
df = df.reset_index(drop=True)
df

,text_clean,label
0,i have the same problem as here but i update f...,1
1,the time estimate and time remaining values on...,0
2,note this bug report is for confluence server ...,1
3,steps to reproduce get a rapid view which is r...,0
4,i used the feed builder to create an rss feed ...,0
...,...,...
28851,issue summary connect app ui elements with the...,1
28852,issue summary group picker with a restricted c...,0
28853,navigation navigate to file status working cop...,1
28854,note this bug report is for confluence cloud u...,0


In [6]:
null_rows = df[df['text_clean'].isnull()]
null_rows

,text_clean,label


In [7]:
df.label.value_counts().to_frame()[:50]
value_counts_df = df['label'].value_counts().to_frame()
value_counts_df

,count
label,
1,14428
0,14428


In [8]:
# Split dataframe into three parts: training, validation and testing.
# Currently 80/10/10 split.
def train_validate_test_split(df, train_percent=.8, validate_percent=.1, seed=42):
    np.random.seed(seed)
    # Shuffle index of dataframe
    perm = np.random.permutation(df.index)
    
    df_length = len(df.index)
    
    # Number of row in training set
    train_end = int(train_percent * df_length)
    # Number of rows in validate set
    validate_end = int(validate_percent * df_length) + train_end
    
    # From start to train end
    train = df.iloc[perm[:train_end]]
    # From train_end to validate_end
    validate = df.iloc[perm[train_end:validate_end]]
    # From validate to the last row in dataframe.
    test = df.iloc[perm[validate_end:]]
    return train, validate, test

In [9]:
# Split dataframe into three parts: training, validation and testing.
train , validate , test = train_validate_test_split(df)

In [10]:
# Convert from Pandas DataFrame to Hugging Face datasets
tds = Dataset.from_pandas(train)
vds = Dataset.from_pandas(validate)
test_ds = Dataset.from_pandas(test)

ds = DatasetDict()

ds["test"] = test_ds
ds["train"] = tds
ds["validate"] = vds

ds

DatasetDict({
    test: Dataset({
        features: ['text_clean', 'label', '__index_level_0__'],
        num_rows: 2887
    })
    train: Dataset({
        features: ['text_clean', 'label', '__index_level_0__'],
        num_rows: 23084
    })
    validate: Dataset({
        features: ['text_clean', 'label', '__index_level_0__'],
        num_rows: 2885
    })
})

In [11]:
# Uncomment this if not logged in.
#notebook_login()

In [12]:
# Push to Hugging Face Hub
ds.push_to_hub(name)

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/24 [00:00<?, ?ba/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]